In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'd:\\text\\text-summarizer-project'

In [4]:
from pathlib import Path

In [5]:
from dataclasses import dataclass
@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer: Path

In [6]:
from src.text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml,create_dirs

In [7]:

class ConfigurationManager:
    def __init__(
        self,
        config_path=CONFIG_FILE_PATH,
        param_path=PARAM_FILE_PATH
    ):
        self.config=read_yaml(config_path)
        self.params=read_yaml(param_path)
        create_dirs([self.config.artifacts_root])
    def get_data_transformation_config(self)->DataTransformationConfig:
        config=self.config.data_transformation
        create_dirs([config.root_dir])
        data_transformation_config=DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer=config.tokenizer
        )
        return data_transformation_config

In [8]:
from text_summarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset,load_from_disk

c:\conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
class DataTransformation:
    def __init__(self,config:DataTransformationConfig):
        self.config=config
        self.tokenizer=AutoTokenizer.from_pretrained(config.tokenizer)
    def convert_examples_to_features(self,examples):
        input_encoding=self.tokenizer(examples['dialogue'],max_length=1024,truncation=True)
        with self.tokenizer.as_target_tokenizer():
            target_encoding=self.tokenizer(examples['summary'],max_length=128,truncation=True)
        return {
            'input_ids':input_encoding['input_ids'],
            'attention_mask':input_encoding['attention_mask'],
            'labels':target_encoding['input_ids']
        }
    def convert(self):
        samsum_dataset=load_from_disk(self.config.data_path)
        samsum_dataset_trans=samsum_dataset.map(self.convert_examples_to_features,batched=True)
        samsum_dataset_trans.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

In [14]:
try:
    config=ConfigurationManager()
    data_transformation_config=config.get_data_transformation_config()
    data_transformation=DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2023-08-23 17:11:58,593: INFO: common Successfully loaded config\config.yaml ,yaml file]
[2023-08-23 17:11:58,600: INFO: common Successfully loaded params.yaml ,yaml file]


Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 45381.01 examples/s]
